In [ ]:
import pandas as pd
import lecilab_behavior_analysis.utils as utils
from pathlib import Path
%load_ext autoreload
%autoreload 2


In [ ]:
# mouse = "mouse2"
# df = utils.load_example_data(mouse)

In [ ]:
# load data from cluster
tv_projects = utils.get_server_projects()
print(tv_projects)

In [ ]:
# see the available animals
animals = utils.get_animals_in_project(tv_projects[1])
print(animals)

In [ ]:
# download the data for a specific animal

# mouse = "mouse2"
# df = utils.load_example_data(mouse)

mouse = "ACV008"
local_path = Path(utils.get_outpath()) / Path(tv_projects[1]) / Path("sessions") / Path(mouse)
# create the directory if it doesn't exist
local_path.mkdir(parents=True, exist_ok=True)
# download the session data
utils.rsync_session_data(
    project_name=tv_projects[1],
    animal=mouse,
    local_path=str(local_path),
    credentials=utils.get_idibaps_cluster_credentials(),
)

In [ ]:
# load the data
df = pd.read_csv(local_path / Path(f'{mouse}.csv'), sep=";")

In [ ]:
from lecilab_behavior_analysis.figure_maker import subject_progress_figure
fig = subject_progress_figure(df, perf_window=100, summary_matrix_plot=False)

In [ ]:
import lecilab_behavior_analysis.df_transforms as dft
import lecilab_behavior_analysis.plots as plots
df_ch = dft.get_center_hold_df(dft.analyze_df(df))
plots.plot_mean_and_cis_by_date(df_ch, item_to_show="number_of_pokes", group_trials_by="year_month_day", color='tab:green')
plots.plot_mean_and_cis_by_date(df_ch, item_to_show="hold_time", group_trials_by="year_month_day", color='tab:red')

In [ ]:
from lecilab_behavior_analysis.df_transforms import add_trial_of_day_column_to_df, add_day_column_to_df
import matplotlib.pyplot as plt

df = add_day_column_to_df(df)
df = add_trial_of_day_column_to_df(df)

tdf = df[df["year_month_day"] == "2025-06-21"]

tdf["port2_holds"] = tdf.apply(lambda row: utils.get_trial_port_hold(row, 2), axis=1)
fig, axs = plt.subplots(2, 1, figsize=(10, 5))
ax1 = plots.plot_number_of_pokes_histogram(tdf, port_number=2, ax=axs[0])
ax2 = plots.plot_port_holding_time_histogram(tdf, port_number=2, ax=axs[1])

plt.show()

In [ ]:
# # load the data if cluster is down
# mouse_path = utils.get_outpath() + "/visual_and_COT_data/sessions/ACV008/ACV008.csv"
# df = pd.read_csv(mouse_path, sep=";")

In [ ]:
from lecilab_behavior_analysis.figure_maker import session_summary_figure
from lecilab_behavior_analysis.df_transforms import add_trial_of_day_column_to_df, add_day_column_to_df
# select the session you want to plot
date = "2025-06-26"
df = add_day_column_to_df(df)
df = add_trial_of_day_column_to_df(df)
sdf = df[df["year_month_day"] == date]
fig = session_summary_figure(sdf, perf_window=50, width=15, height=10)

In [ ]:
import lecilab_behavior_analysis.plots as plots
import lecilab_behavior_analysis.df_transforms as dft
import matplotlib.pyplot as plt
import numpy as np
sdf["total_trial"] = np.arange(1, sdf.shape[0] + 1)
# find the index of the session changes
session_changes = sdf[sdf.session != sdf.session.shift(1)].index
sdf = dft.calculate_time_between_trials_and_reaction_time(sdf)

# test = plots.plot_time_between_trials_and_reaction_time(sdf, session_changes=session_changes)

reaction_time_image = plots.rasterize_plot(plots.plot_time_between_trials_and_reaction_time(sdf, session_changes=session_changes), dpi=300)
fig, ax = plt.subplots(figsize=(10, 5))
ax.imshow(reaction_time_image, aspect='auto')
plt.axis('off')
plt.show()

In [ ]:
from lecilab_behavior_analysis.plots import plot_time_between_trials_and_reaction_time
from lecilab_behavior_analysis.df_transforms import calculate_time_between_trials_and_reaction_time
session = 28
sdf = df[df["session"] == session]
ssdf = calculate_time_between_trials_and_reaction_time(sdf)
ssdf


In [ ]:
ssdf.Reaction_Time.plot(kind="hist", bins=100)